In [1]:
import random

# Modules to import

from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationalRetrievalChain, RetrievalQA
from langchain.document_loaders import PyPDFLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.memory import ConversationBufferMemory
from langchain.text_splitter import RecursiveCharacterTextSplitter

from langchain.vectorstores.faiss import FAISS

# Get OpenAI API KEY

In [1]:
import os
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv()) # read local .env file
api = os.getenv('OPENAI_API_KEY_H')

# Split Chunk

In [3]:
r_splitter = RecursiveCharacterTextSplitter(
    chunk_size=600,
    chunk_overlap=100,
    separators=["\n\n", "\n", "(?<=\. )", " ", ""]
)

docs = []
for loader in os.listdir("docs"):
    if loader is not None :
        try :
            docs.extend(PyPDFLoader("docs/"+loader).load())
        except Exception as e1 :
            print(f" [ Documents ERROR ] : {e1}")
    else : break

In [4]:
docs[:2]

[Document(page_content='1-72 สารบัญ ภาพโดยรวม/คู่มืออย่างง่าย        1 ข้อมูลทั่วไป          2 การล็อกและการปลดล็อก        3 เบาะนั่งและเข็มขัดนิรภัย         4 แผงหน้าป้ดและอุปกรณ์ควบคุม        5 การสตาร์ทและการขับขี่         6 การใช้อุปกรณ์อํานวยความสะดวก       7 เมื่อเกิดเหตุฉุกเฉิน         8 การดูแลรักษาสภาพรถ         9 การบํารุงรักษา          10 ข้อมูลจําเพาะ          11 ', metadata={'source': 'docs/Text_PDF_.pdf', 'page': 0}),
 Document(page_content='แผงหน้าป้ดและอุปกรณ์ควบคุม(ด้านคนขับ) ภาพโดยรวม/คู่มืออย่างง่าย1-11 - สวิตช์ที่ป้ดนํ้าฝนและฉีดนํ้าล้างกระจกหน้า5-45 - แผงหน้าป้ดหน้า5-2 - สวิตช์ควบคุมความสว่างของแผงหน้าป้ดหน้า5-2 - สวิตช์ระบบควบคุมเสถียรภาพการทรงตัว(ASC)OFFหน้า6-70 - สวิตช์เซ็นทรัลล็อกหน้า3-15 - สวิตช์ควบคุมหน้าต่างไฟฟ้าหน้า3-19 - สวิตช์ล็อกหน้า3-20 - สวิตช์ไฟหน้าและไฟสูง-ตํ่าหน้า5-40 - สวิตช์ระบบปรับไฟสูงอัตโนมัติ(AHB)หน้า5-43 - คันสวิตช์ไฟเลี้ยวหน้า5-44 - สวิตช์ไฟตัดหมอกด้านหน้าหน้า5-45 - สวิตช์หน้าจอแสดงข้อมูลรวมหน้า5-4 - สวิตช์ควบคุมเครื่องเสียงบนพวงมาลัยให้ดูคู่

# Vector Database

In [5]:
splits = r_splitter.split_documents(docs)

db = FAISS.from_documents( splits, OpenAIEmbeddings( api_key = api ) )

In [6]:
splits[random.randint(0,len(splits))]

Document(page_content=': กะพริบ (ตัวแสดงโหมดการขับเคลื่อน4 ล้อกะพริบด้วย) : สว่าง : ดับ “4H”  “4HLc” \n: กะพริบ (ตัวแสดงโหมดการขับเคลื่อน 4 ล้อกะพริบด้วย) : สว่าง : ดับ ตําแหน่งปุ่มเลือกการขับเคลื่อน 4 ล้อตัวแสดง2Hกําลังเปลี่ยนโหมดการขับเคลื่อน 4 ล้อ4H\nตําแหน่งปุ่มเลือกการขับเคลื่อน 4 ล้อตัวแสดง4Hกําลังเปลี่ยนโหมดการขับเคลื่อน 4 ล้อ4HLc', metadata={'source': 'docs/Text_PDF_.pdf', 'page': 196})

# CHAT Method

In [7]:
# Memory

memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True,
    output_key='answer',
)

In [8]:
model_lists = [ "gpt-3.5-turbo", 'gpt-4-1106-preview', "gpt-4", ]

llm = ChatOpenAI( api_key = api, model_name = model_lists[0], temperature = 0 )

In [34]:
# qa = RetrievalQA.from_chain_type(
#     llm,
#     retriever=db.as_retriever(search_type="similarity", search_kwargs={"k": 1}),
#     chain_type="map_reduce",
#     return_source_documents = True,
# )

In [12]:
memory.clear()

In [9]:
qa = ConversationalRetrievalChain.from_llm(
            llm = llm,
            retriever = db.as_retriever(search_type="similarity", search_kwargs={"k": 3}),
            memory = memory,
            return_source_documents = True,
            return_generated_question = True,
        )

## Get answer (Testing)

In [14]:
ran = splits[random.randint(0,len(splits))]

prompt = f"""
context : '{ran.page_content}'
Create 1 question using only the context provided.
End each question with a '?' character.
You must answer the question by refer to the context as much as you can.
Separate question/answer by a newline.
the question and the answer should be thai language.
"""

# Test
result = qa({"question": prompt})

print(f"context : {(ran.page_content)[:100]}\npage : {(ran.metadata)['page']+1}\n\n")

print(f"{result.keys()}\n\n")
print(f"Question :\n{result['generated_question']}\n\nAnswer :\n{result['answer']}\n\nSource :\nPage : {(result['source_documents'])[0].metadata}\nContent : {(result['source_documents'])[0].page_content}")

context : หมายเหตุ  
● หากคุณยังคงกดสวิตช์  “ASC OFF” ค้างไว้หลังจากที่ฟังก์ชันควบคุมเสถียรภาพปิดไปแล้ว  “ฟังก
page : 210


dict_keys(['question', 'chat_history', 'answer', 'source_documents', 'generated_question'])


Question :

context : 'หมายเหตุ  
● หากคุณยังคงกดสวิตช์  “ASC OFF” ค้างไว้หลังจากที่ฟังก์ชันควบคุมเสถียรภาพปิดไปแล้ว  “ฟังก์ชันป้องกันการทํางานผิดพลาด ” 
จะทํางานและเปิดฟังก์ชันควบคุมเสถียรภาพอีกครั้ง  
● แม้จะปิดฟังก์ชันควบคุมเสถียรภาพไปแล้ว  แต่ฟังก์ชันนี้อาจกลับมาทํางานได้โดยขึ้นอยู่กับการเคลื่อนไหวของรถ  
 
ฟังก์ชันควบคุมการออกตัวและการเข้าโค้ง  
E00617001048 
ฟังก์ชันควบคุมการออกตัวและการเข้าโค้งป้องกันไม่ให้ล้อหมุนมากเกินไปบนถนนลื่นและช่วยให้รถที่หยุดอยู่ออกตัวได้  
นอกจากนี้ยังให้กําลังการขับเคลื่อนที่เพียงพอและทําให้การหมุนพวงมาลัยมีประสิทธิภาพเมื่อรถเลี้ยวขณะเหยียบคันเร่ง 
 
ข้อควรระวัง'
Create 1 question using only the context provided.
End each question with a '?' character.
You must answer the question by refer to the context as much as you can.
Separate quest

In [15]:
((result['source_documents'])[0].metadata)['page']+1

210

In [14]:
# del result

# Bulid dataset

In [1]:
import pandas as pd

df = pd.read_csv("rerank_dataset.csv", index_col=0)

df.tail(3)

,questions,answer,doc_page,doc_content,doc_source
90,ต้องทำอย่างไรเมื่อความเร็วรถลดลงต่ำกว่าประมาณ ...,"เมื่อความเร็วรถลดลงต่ำกว่าประมาณ 30 กม./ชม., ค...",127,แผงหน้าปัดและอุปกรณ์ควบคุม 5 \nคันสวิตช์ไฟเล...,docs/Text_PDF_.pdf
91,ระบบ FCM อาจไม่ทำงานในสภาวะใดบ้าง?,ระบบ FCM อาจไม่ทำงานในสภาวะดังต่อไปนี้:\n- รถค...,213,○ ระบบ FCM อาจไม่ทํางานหากรถคันหน้ามีลักษณะแคบ...,docs/Text_PDF_.pdf
92,ตัวแสดงโหมดการขับเคลื่อน 4 ล้อแสดงสถานะการเปลี...,ตัวแสดงโหมดการขับเคลื่อน 4 ล้อจะสว่างตามตำแหน่...,180,ตัวแสดงโหมดการขับเคลื่อน 4 ล้อและตัวแสดงการทํ...,docs/Text_PDF_.pdf


In [17]:
rerank_df = {"questions":[], "answer":[], "doc_page":[], "doc_content":[], "doc_source":[]}

In [18]:
for i in range(51):
    ran = splits[random.randint(0,len(splits))]
    prompt = f"""
context : '{ran.page_content}'
Create 1 question using only the context provided.
End each question with a '?' character.
You must answer the question by refer to the context as much as you can.
Separate question/answer by a newline.
the question and the answer should be thai language.
"""
    try :
        result = qa({"question": prompt})
        
        rerank_df["answer"].append(result['answer'])
        rerank_df["questions"].append(result['generated_question'])
        rerank_df["doc_content"].append(ran.page_content)
        rerank_df["doc_page"].append( (ran.metadata)['page']+1 )
        rerank_df["doc_source"].append( (ran.metadata)['source'] )
        
    except :
        memory.clear()
        result = qa({"question": prompt})
 

In [19]:
r_df = pd.DataFrame.from_dict(rerank_df)

print(f"number of dataset : {len(r_df)}\n\n")
r_df.tail(3)

number of dataset : 42




,questions,answer,doc_page,doc_content,doc_source
39,ต้องทำอย่างไรเมื่อความเร็วรถลดลงต่ำกว่าประมาณ ...,"เมื่อความเร็วรถลดลงต่ำกว่าประมาณ 30 กม./ชม., ค...",127,แผงหน้าปัดและอุปกรณ์ควบคุม 5 \nคันสวิตช์ไฟเล...,docs/Text_PDF_.pdf
40,ระบบ FCM อาจไม่ทำงานในสภาวะใดบ้าง?,ระบบ FCM อาจไม่ทำงานในสภาวะดังต่อไปนี้:\n- รถค...,213,○ ระบบ FCM อาจไม่ทํางานหากรถคันหน้ามีลักษณะแคบ...,docs/Text_PDF_.pdf
41,ตัวแสดงโหมดการขับเคลื่อน 4 ล้อแสดงสถานะการเปลี...,ตัวแสดงโหมดการขับเคลื่อน 4 ล้อจะสว่างตามตำแหน่...,180,ตัวแสดงโหมดการขับเคลื่อน 4 ล้อและตัวแสดงการทํ...,docs/Text_PDF_.pdf


In [20]:
df = pd.concat([df, r_df], 
                  ignore_index = True)

print(f"number of dataset : {len(df)}\n\n")
display(df.tail(3))

number of dataset : 93




,questions,answer,doc_page,doc_content,doc_source
90,ต้องทำอย่างไรเมื่อความเร็วรถลดลงต่ำกว่าประมาณ ...,"เมื่อความเร็วรถลดลงต่ำกว่าประมาณ 30 กม./ชม., ค...",127,แผงหน้าปัดและอุปกรณ์ควบคุม 5 \nคันสวิตช์ไฟเล...,docs/Text_PDF_.pdf
91,ระบบ FCM อาจไม่ทำงานในสภาวะใดบ้าง?,ระบบ FCM อาจไม่ทำงานในสภาวะดังต่อไปนี้:\n- รถค...,213,○ ระบบ FCM อาจไม่ทํางานหากรถคันหน้ามีลักษณะแคบ...,docs/Text_PDF_.pdf
92,ตัวแสดงโหมดการขับเคลื่อน 4 ล้อแสดงสถานะการเปลี...,ตัวแสดงโหมดการขับเคลื่อน 4 ล้อจะสว่างตามตำแหน่...,180,ตัวแสดงโหมดการขับเคลื่อน 4 ล้อและตัวแสดงการทํ...,docs/Text_PDF_.pdf


In [4]:
df.to_csv("rerank_dataset.csv")

# Metadata enrichment
- Using keyBERT
- Using PyThaiNLP with keyBERT

In [1]:
# ! pip install keybert
# ! pip install keybert[flair] keybert[gensim] keybert[spacy] keybert[use]

In [2]:
# ! pip install --upgrade pythainlp
# ! pip install attacut

## Dataset optimization

In [2]:
# difference check
def diff_check(a,b):
    for i,j in zip(a,b):
        assert (i == j)

# Check result

def check_r(list):
    return print(f"Number of items in the list : {len(list)}\nexample 1 (first) : {list[0]}\nexample 2 (last) : {list[-1]}")

# Clean text

def clean_text(text):
    text = text.replace(' ','').replace('(','')\
        .replace(')','').replace('?','').replace('●','')\
        .replace('/','').replace('-','').replace('•','')\
        .replace('○','').replace('\\','').replace('"','')\
        .replace('■','').replace('”','').replace('“','')\
        .replace(':','').replace(',',':')
    return text


In [3]:
label_from_doc = list()
label_from_questions = list()

for i,j in zip(df['doc_content'],df['questions']) :
    label_from_doc.append(i)
    label_from_questions.append(j)

In [4]:
diff_check(label_from_doc,df['doc_content'])
diff_check(label_from_questions,df['questions'])

## keyBERT
- extracting keywords

In [5]:
# from keybert import KeyBERT

# doc = """
#          Supervised learning is the machine learning task of learning a function that
#          maps an input to an output based on example input-output pairs. It infers a
#          function from labeled training data consisting of a set of training examples.
#          In supervised learning, each example is a pair consisting of an input object
#          (typically a vector) and a desired output value (also called the supervisory signal).
#          A supervised learning algorithm analyzes the training data and produces an inferred function,
#          which can be used for mapping new examples. An optimal scenario will allow for the
#          algorithm to correctly determine the class labels for unseen instances. This requires
#          the learning algorithm to generalize from the training data to unseen situations in a
#          'reasonable' way (see inductive bias).
#       """
# kw_model = KeyBERT(model="paraphrase-multilingual-MiniLM-L12-v2")
# keywords = kw_model.extract_keywords(doc, keyphrase_ngram_range=(1, 1), stop_words=None)

In [6]:
# keywords

[('supervised', 0.5804),
 ('supervisory', 0.5156),
 ('learning', 0.5126),
 ('training', 0.4726),
 ('input', 0.3581)]

In [7]:
# # keywords by doc_content

# keywords = kw_model.extract_keywords(
#                                     label_from_doc,
#                                      keyphrase_ngram_range=(1, 1),
#                                      stop_words = None,
#                                      use_mmr = True,
#                                      vectorizer = None,
#                                      )

# check_r(keywords)

Number of items in the list : 93
example 1 (first) : [('กระจกยกเว', 0.4142), ('โดยสารจะไม', 0.3813), ('องแบตเตอร', 0.26), ('และไฟแสดงจะด', 0.2309), ('พยายามอย', 0.1776)]
example 2 (last) : [('4wd', 0.5351), ('4h', 0.4199), ('e00606001774', 0.269), ('วแสดงความเร', 0.2323), ('2h', 0.1909)]


In [26]:
# # keywords by question

# keywords = kw_model.extract_keywords(label_from_questions,
#                                      keyphrase_ngram_range=(1, 1),
#                                      stop_words=None,
#                                      )

# check_r(keywords)

Number of items in the list : 93
example 1 (first) : [('างไร', 0.4069), ('ตช', 0.3885), ('สว', 0.2757), ('อกทำงานอย', 0.2298)]
example 2 (last) : [('วแสดงโหมดการข', 0.3286), ('อแสดงสถานะการเปล', 0.3193), ('างไร', 0.209), ('อน', 0.1939), ('ยนโหมดอย', 0.1609)]


original ```keyBERT``` does not work for Thai language

## PyThaiNLP with KeyBERT
- extracting keywords

In [7]:
from pythainlp.summarize.keybert import KeyBERT as thai_kb

kb = thai_kb()

def get_keywords(lists):
    global kb
    keywords=[]
    for i in lists:
        kw = kb.extract_keywords(i,
                                tokenizer="attacut",
                                max_keywords=5,
                                return_similarity=True
                                )
        keywords.append(kw)
    return keywords

In [8]:
# keywords by question

keywords = get_keywords(label_from_questions)
check_r(keywords)

[W NNPACK.cpp:64] Could not initialize NNPACK! Reason: Unsupported hardware.


Number of items in the list : 93
example 1 (first) : [('ล็อกทำ', 0.67616502249311), ('สวิตช์ล็อก', 0.5886666139639188), ('?', 0.38394343227401284), ('งาน', 0.3561380618379986), ('ไร?', 0.28996203712898116)]
example 2 (last) : [('โหมดอย่าง', 0.39368097804398205), ('?', 0.35116622377584567), ('ไร?', 0.3459296969723583), ('แสดงโหมด', 0.33969007330578915), ('สถานะการ', 0.3252000116851641)]


In [9]:
# keywords by chunk

keywords_d = get_keywords(label_from_doc)
check_r(keywords_d)

Number of items in the list : 93
example 1 (first) : [('ดับเบิลแค็บ)', 0.5401025750345892), ('งาน', 0.3737799935672376), ('ขับ', 0.3660317295125938), ('ขับการ', 0.3569568996731559), ('ซํ้า', 0.35637276798135975)]
example 2 (last) : [('E00606001774', 0.4254320220657912), ('งาน', 0.3541632610704103), ('2WD/4WD  ', 0.34959007610036363), ('จะสว่าง', 0.34245305726227326), ('เร็วตํ่า ', 0.3363391918722709)]


## Clean Results DATA

In [10]:
col = {'questions':[],'label':[]}
for i in range( len(df) ) :
    row = []
    for j in keywords_d[i] :
        temp = clean_text(j[0])
        row.append( temp )
    col['questions'].append(df['questions'][i])
    col['label'].append(row)

col.keys()

# to dataframe

col = pd.DataFrame.from_dict(col)
display(col.tail(3))

dict_keys(['questions', 'label'])

In [11]:
# join column

df = df.merge(col, how='left', on='questions')

df.tail(3)

,questions,label
0,สวิตช์ล็อกทำงานอย่างไร?,"[ดับเบิลแค็บ, งาน, ขับ, ขับการ, ซํ้า]"
1,การขับขี่ขึ้นทางชัน E00647000125 ต้องตั้งค่าปุ...,"[SuperSelect, 4WD, สลับ, ความสามารถ, E00647000..."
2,อุปกรณ์ภายนอกที่อยู่ด้านหน้าของรถมีอะไรบ้าง?,"[44:10, 24:10, 44:10, 24:10, 24:10]"
3,รถรุ่นที่ติดตั้งการเตือนคาดเข็มขัดนิรภัยสำหรับ...,"[งาน, สําหรับเบาะ, ถูก, นั่งที่, บาน]"
4,บนถนนที่ปกคลุมด้วยหิมะหรือลื่นอาจมีความแตกต่าง...,"[, ถนน, ปกคลุม, การณ์แนวทาง, แสดงไม่]"
...,...,...
88,"เมื่อเลือกโหมด ""NORMAL"" ของโหมดการขับเคลื่อน ต...","[อ้างอิง, งาน, รายการตัว, NORMAL, NORMAL]"
89,"ต้องดูเรื่อง ""การใช้อุปกรณ์อำนวยความสะดวก"" หน้...","[T00UMA000P_OM_THAI.indb, _T00UMA000P_OM_, T00..."
90,ต้องทำอย่างไรเมื่อความเร็วรถลดลงต่ำกว่าประมาณ ...,"[ลดลงตํ่ากว่า, E00536800021, งาน, โปรด, อุปกรณ..."
91,ระบบ FCM อาจไม่ทำงานในสภาวะใดบ้าง?,"[งาน, อาจทํา, ระหว่างรถ, อาจตรวจ, จับรถ]"


,questions,answer,doc_page,doc_content,doc_source,label
90,ต้องทำอย่างไรเมื่อความเร็วรถลดลงต่ำกว่าประมาณ ...,"เมื่อความเร็วรถลดลงต่ำกว่าประมาณ 30 กม./ชม., ค...",127,แผงหน้าปัดและอุปกรณ์ควบคุม 5 \nคันสวิตช์ไฟเล...,docs/Text_PDF_.pdf,"[ลดลงตํ่ากว่า, E00536800021, งาน, โปรด, อุปกรณ..."
91,ระบบ FCM อาจไม่ทำงานในสภาวะใดบ้าง?,ระบบ FCM อาจไม่ทำงานในสภาวะดังต่อไปนี้:\n- รถค...,213,○ ระบบ FCM อาจไม่ทํางานหากรถคันหน้ามีลักษณะแคบ...,docs/Text_PDF_.pdf,"[งาน, อาจทํา, ระหว่างรถ, อาจตรวจ, จับรถ]"
92,ตัวแสดงโหมดการขับเคลื่อน 4 ล้อแสดงสถานะการเปลี...,ตัวแสดงโหมดการขับเคลื่อน 4 ล้อจะสว่างตามตำแหน่...,180,ตัวแสดงโหมดการขับเคลื่อน 4 ล้อและตัวแสดงการทํ...,docs/Text_PDF_.pdf,"[E00606001774, งาน, 2WD4WD, จะสว่าง, เร็วตํ่า]"


In [12]:
# save

df.to_csv("rerank_dataset.csv")